In [2]:
# 책 내용 대신 파이썬 기본 smtplib, email, imaplib 모듈 활용
import smtplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import imaplib
from pathlib import Path
import pyinputplus as pyip

In [2]:
# smtplib.SMTP()함수를 통해 smtp 객체 생성(587 포트 활용)
# 587 포트가 안될 경우 smtplib.SMTP_SSL('smtp.gmail.com', 465) 활용
smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
# smtp 객체를 생성한 후에는 ehlo() 메서드를 제일 먼저 호출해야(헬로 도메인)
smtpObj.ehlo()
# 연결을 암호화 하기 위하여 starttls() 메서드 호출 -> SSL을 사용하여 465 포트에 연결했다면 이 단계 건너뛰기 가능
smtpObj.starttls()

(220, b'2.0.0 Ready to start TLS')

In [3]:
# pyinputplus를 활용하여 소스코드에 비밀번호 직접 넣지 않도록 조치
password = pyip.inputStr(prompt='비밀번호를 입력해주세요.: ')
# login 메서드를 이용해 로그인 -> 반환값 235는 인증에 성공했음을 의미
smtpObj.login('kth1149@gmail.com', password)
password = None

비밀번호를 입력해주세요.: 

In [4]:
# MIMEMultipart() 및 MIMEText() 이용하여 MIMEText(메시지) 등 객체 생성
# MIMEText()에 html 문서를 집어넣을 수도 있으며, 그 경우 두 번째 인자로 'html' 넣어줘야 -> HTML 웹에디터 활용 가능
msg = MIMEMultipart()
# msg['subject'], msg['to'], msg['from'], msg['cc'], msg['bcc'] 활용
msg['Subject'] = '파이썬에서 보내는 이메일입니다.'
msg['To'] = 'tkim@iportfolio.co.kr'
msg['From'] = 'kth1149@gmail.com'
msg['cc'] = 'kth1149@gmail.com'
body = '파이썬에서 보내는 이메일입니다.'
# attach 메서드를 활용하여 Multipart()에 MIMEText() 객체 삽입
msg.attach(MIMEText(body))

In [5]:
# sendmail() 메서드를 통해 메일 보내기(보내는 사람, 받는 사람, 메시지 내용 순서의 인자)
# msg.as_string()을 활용하여 설정되었던 'cc' 주소로도 메일을 보낼 수 있음을 확인
smtpObj.sendmail(msg['from'], msg['to'], msg.as_string())

{}

In [6]:
# quit() 메서드를 사용해 연결 종료
smtpObj.quit()

(221,
 b'2.0.0 closing connection a4-20020a170902ecc400b0016d2e772550sm6995609plh.175 - gsmtp')

In [3]:
# imaplib.IMAP4_SSL() 함수를 통해 imap 객체 생성
imapObj = imaplib.IMAP4_SSL('imap.gmail.com')

In [4]:
# pyinputplus를 활용하여 소스코드에 비밀번호 직접 넣지 않도록 조치
password = pyip.inputStr(prompt='비밀번호를 입력해주세요.: ')
# login 메서드를 이용해 로그인 -> Success는 인증에 성공했음을 의미
imapObj.login('kth1149@gmail.com', password)
password = None

비밀번호를 입력해주세요.: 

In [5]:
# select() 메서드를 통하여 메일함 지정 -> 'INBOX' 등
# readonly=True 인자를 통해 읽기 전용으로 열기도 가능
imapObj.select('INBOX', readonly=True)

('OK', [b'24322'])

In [6]:
# 메일함의 리스트 확인
imapObj.list()

('OK',
 [b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasNoChildren) "/" "Sent"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\Flagged \\HasNoChildren) "/" "[Gmail]/&vMTUXNO4ycDVaA-"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/&vPSwuNO4ycDVaA-"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/&wqTTONVo-"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/&x4TC3Lz0rQDVaA-"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/&yATMtLz0rQDVaA-"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/&yRHGlA-"',
  b'(\\HasNoChildren \\Trash) "/" "[Gmail]/&1zTJwNG1-"'])

In [7]:
# search() 메서드 활용 -> 첫 번째 인자는 charset, 두 번째 인자는 찾고자 하는 내용
# status와 result(uid의 리스트 형태)를 튜플로 반환
status, result = imapObj.search('utf-8', '(FROM "kth1149@gmail.com")')
# result는 리스트 형태이므로, index 0을 지정 후 split()으로 새로운 리스트로 분류
result = result[0].split()
result[:2]

[b'936', b'993']

In [8]:
# 가장 최근 uid를 선택
recent = result[-5]
# fetch() 메서드를 활용하여 확인 -> 첫 번째 인자는 확인하고자 하는 uid, 두 번째 인자는 '(RFC822)'로 고정
# 마찬가지로 status와 result(인코딩 이전 메시지 형태)를 튜플로 반환
status, result = imapObj.fetch(recent, '(RFC822)')

In [9]:
# result[0][1](이게 뭔지는 모르겠으나.. 필요한 정보만을 따로 추출한 것으로 보임)를 raw로 지정
raw = result[0][1]

In [10]:
# email 모듈 활용하여 진행
# email.message_from_bytes() 활용 -> 디코딩을 진행하지 않았으므로
email_msg = email.message_from_bytes(raw)
# keys() 메서드를 통해 어떠한 키들이 있는지 확인 -> 즉, email_msg는 딕셔너리의 형태
email_msg.keys()

['Return-Path',
 'Received',
 'Message-ID',
 'Date',
 'From',
 'X-Google-Original-From',
 'Content-Type',
 'MIME-Version',
 'To',
 'cc',
 'Subject']

In [11]:
# 수신인, 발신인, 제목, 일시 확인
email_msg['To'], email_msg['From'], email_msg['Subject'], email_msg['Date']

('kth1149@gmail.com',
 'kth1149@gmail.com',
 '[IMPORTANT] Jun 2014 dues unpaid.',
 'Thu, 28 Jul 2022 07:22:23 -0700 (PDT)')

In [16]:
# 메일 본문 내용을 get_payload() 메서드를 통해 확인
print(email_msg.get_payload())

In [121]:
# imap에서 로그아웃 진행
imapObj.logout()

('BYE', [b'LOGOUT Requested'])